In [1]:
# Libraries import

import os, gc
import numpy as np
#import pickle
import joblib
import pandas as pd
import sys

from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

np.set_printoptions(suppress=True, precision = 3)

# Notes 

## Model architecture 
We train a random forest classifier, using a 5-fold cross-validation. The validation folds are designed using over-sampling so that in each fold, the validation set contains at least one tumor from each class, but possibly more so that each class has at least 15% of its data in validation. 

We select the most important features from the random forest model for experimental validation. We also use them to train a support vector classifier model, to boost the quality of the model. 

In [2]:
"""
DATA DESCRIPTION
Indicate the file names for the pre-processed data and for the tumor/non-tumor map. 
The pre-processed data is a list of dictionaries, one for each sample, each dictionary has the 2D x 1738 data
plus its methylation label, sample name, patient ID, , t-SNE and PCA decompositions (datastructure described below).
The tumor/non-tumor map is a list of numpy arrays, one for each sample, of the same 2D shape as the 
corresponding sample, coming in the same order as the samples in the pre-processed file. 
Each entry in the 2D array is 0/1, with 1 indicating that the corresponding spot in the pre-processed data 
was labeled as tumor. 
The map is to be used as filter: only those spot labeled as tumor are to be analysed further. 
We also use the map to extract the most discriminative features between tumor and non-tumor. 
"""

PREPROC_DATA_PATH = "../data/preprocessed.npy"
TUMOR_MAP_PATH = "../data/tumor_maps.npy"
SIX_CLASSIFIER_PATH = "./SixMethylModels/"

# Make the directory if it doesn't exist
if not os.path.exists(SIX_CLASSIFIER_PATH[2:-1]):
    os.mkdir(SIX_CLASSIFIER_PATH[2:-1])

SAMPLE_DICT_DATA_KEY = "data"
SAMPLE_DICT_NAME_KEY = "name"
SAMPLE_DICT_CLASS_ID_KEY = "class_id"
SAMPLE_DICT_PATIENT_ID_KEY = "patient_id"
SAMPLE_DICT_TSNE_KEY = "tsne"
SAMPLE_DICT_PCA_KEY = "pca"

# Define the samples to be excluded from the analysis (use their full name)
ignore = ["HF-1887_via-t_2_1.h5_3"]

In [3]:
"""
CROSS-VALIDATION DESIGN: 5-fold
All samples (except 1887) included. Assigned to the CV folds in order of their names, 
trying to have about the same amount of validation data in each fold.
Date: February 2023. 
"""

CV5folds=[ 
    [#Cv fold 1
        #LGm1 
        "HF-448_V5B_1.h5_3",
        #LGm2
        "HF-305_v4b_1_1.h5_6", "HF-615_V5BB_1.h5_9",  
        #LGm3
        "HF-2104_#5_1.h5_0", "HF-2104_#9_1.h5_1", "HF-2104_V1T_1.h5_2",
        #LGm4
        "HF-442_V4BB_1.h5_12", "HF-1002_V1AT_1.h5_0", "HF-1002_V2AT_1.h5_1", 
        #LGm5
        "HF-682_V3AT_1.h5_9", "HF-682_V3BB_1.h5_10", "HF-894_9_1.h5_11", "HF-894_V1BB_1.h5_12", 
        #LGm6
        "HF-592_V3T_1.h5_4", 
    ],
    [#Cv fold 2
        #LGm1 
        "HF-868_1_2.h5_4",
        #LGm2
        "HF-901_V2T_2.h5_10", "HF-960_VIAT_2.h5_11", 
        #LGm3
        "HF-2614_V1B_1.h5_3", 
        #LGm4
        "HF-1825_V2B_1.h5_2", "HF-2102_V2BB_1.h5_3", "HF-2102_V3AM_1.h5_4", "HF-2102_V3AM_2.h5_5", 
        #LGm5
        "HF-988_V1-T_1.h5_13", "HF-988_V1B_1.h5_14", "HF-1043_V1AM_1.h5_0", 
        #LGm6
        "HF-2106_V3AM_1.h5_0", 
    ], 
    [#Cv fold 3
        #LGm1 
        "HF-1293_13_1.h5_0",
        #LGm2
        "HF-1010_V1T_1.h5_0", "HF-1016_IAT_2.h5_1", "HF-1334_V58-B_2_1.h5_2", 
        #LGm3
        "HF-2849_VIT2_1.h5_4", "HF-2849_VIT2_1.h5_5", "HF-2849_VIT2_2.h5_6", "HF-2849_VIT_2_new2021.h5_7",
        #LGm4
        "HF-2454_V1AT_1.h5_6", "HF-2548_V1T_1.h5_7", 
        #LGm5
        "HF-1086_#1_1.h5_1", "HF-2355_V2AM_1.h5_2", 
        #LGm6
        "HF-2493_V1T_1.h5_1", "HF-2493_V1T_2.h5_2", 
    ], 
    [#Cv fold 4
        #LGm1 
        "HF-1295_V3AM_2.h5_1",
        #LGm2
        "HF-2070_V1T_1.h5_4", "HF-2776_V2B_2.h5_5",        
        #LGm3 
        "HF-2852_VIT_2_2.h5_8", 
        #LGm4
        "HF-2715_VIL_1.h5_8", "HF-2802_V3T_1.h5_9", 
        #LGm5
        "HF-2485_V1B_1.h5_3", "HF-2600_V1B_1.h5_4", "HF-2608_V1T_1.h5_5", 
        #LGm6
        "HF-2544_V1B_1.h5_3", 
    ],
    [#Cv fold 5
        #LGm1 
        "HF-2534_V2B_1.h5_2",
        #LGm2
        "HF-3271_VIB_2.h5_7", "HF-3337_V3T_1.h5_8", 
        #LGm3 same as fold 1
        "HF-2104_#5_1.h5_0", "HF-2104_#9_1.h5_1", "HF-2104_V1T_1.h5_2",
        #LGm4
        "HF-2876_V1T_1.h5_10", "HF-2898_V1T_1.h5_11",
        #LGm5
        "HF-2619_V1T_1.h5_6", "HF-2619_V4T_1.h5_7", "HF-2666_V2B_1.h5_8",
        #LGm6 same as fold 1
        "HF-592_V3T_1.h5_4", 
    ],
]

In [4]:
def reset_seed(SEED = 0):
    os.environ['PYTHONHASHSEED']=str(SEED)
    np.random.seed(SEED)
reset_seed(2022)

In [5]:
def CrossValidation(data, labels, CV_folds=CV5folds, verbose=False):
    """ 
    This is the cross-validation training. 
    In each CV fold, the model is first trained, then the data reduced to its top 20 features
    and the model is re-trained with an SVC. 
    This is in the hope of reducing the noise and getting a better fit.
    We save all RF and SVC models, as well as their metrics. 
    """
    
    results = []  
    RFmetrics=[]
    SVCmetrics=[]
    index = 0
    
    # Loop through the CV folds
    for valid_fold in CV_folds:
        
        index = index+1
        print("Validation fold", index)
        CV_fold_name=str(index)

        # Get the train/validation data for this CV fold
        train_X=np.empty( (0,data[0][SAMPLE_DICT_DATA_KEY].shape[1]) )
        train_y=np.empty( 0 )
        valid_names=[]
        valid_X=np.empty( (0,data[0][SAMPLE_DICT_DATA_KEY].shape[1]) )
        valid_y=np.empty( 0 )
        for (sample,label) in zip(data,labels): 
            if sample[SAMPLE_DICT_NAME_KEY] in valid_fold:
                valid_names.append(sample[SAMPLE_DICT_NAME_KEY])
                valid_X = np.append(valid_X, sample[SAMPLE_DICT_DATA_KEY], axis=0)
                valid_y = np.append(valid_y, label, axis=0)
            else: 
                train_X = np.append(train_X, sample[SAMPLE_DICT_DATA_KEY], axis=0)
                train_y = np.append(train_y, label, axis=0)
        if len(valid_names) != len(valid_fold):
            print("Error in the design of CV fold (data not found):", valid_fold)
            sys.exit(-1)     
            
        # Create list of indices
        shuffle = np.arange(len(train_X))
        
        # Numpy shuffle method performs shuffle in place
        np.random.shuffle(shuffle)
        
        # Shuffle the training data
        train_X = np.squeeze(train_X[shuffle])
        train_y = np.squeeze(train_y[shuffle])
    
        # Create RF model and train to extract features
        RFclf = RandomForestClassifier(n_estimators = 300,
                            bootstrap = True,
                            max_depth = 10,
                            random_state=0,
                            criterion = "entropy",
                            class_weight = "balanced_subsample",
                            warm_start = False,
                            n_jobs = -1,
                            min_samples_leaf = 50,
                            min_samples_split = 50,
                            max_features = "sqrt"
                            )

        RFclf.fit(train_X, train_y)
        
        # Get accuracy on training and validation data
        acc_train = RFclf.score(train_X, train_y)
        acc_val = RFclf.score(valid_X, valid_y)
        if verbose: 
            print("\t RF model. Accuracy on train and valid: ", acc_train, acc_val)
        
        # Get prediction on the validation data
        pred = RFclf.predict(valid_X)
        
        # Gather RF metrics
        if verbose:
            print("\t Computing the RF model metrics.")
        RFacc = balanced_accuracy_score(valid_y, pred)        
        RFprec = precision_score(valid_y, pred, average = None)
        RFrec = recall_score(valid_y, pred, average = None)
        RFf1 = f1_score(valid_y, pred, average = None)
        RFconf = confusion_matrix(valid_y, pred, normalize='true')
        RFmetrics.append([RFacc, RFprec, RFrec, RFf1, RFconf])
        
        if verbose: 
            print("\t RF score:    ", RFclf.score(valid_X, valid_y))
            print("\t RF accuracy: ", RFacc)
            print("\t RF precision:", RFprec)
            print("\t RF recall:   ", RFrec)
            print("\t RF F1:       ", RFf1)
            print("\t RF confusion matrix:\n", RFconf)           
        
        # Save the model
        joblib.dump(RFclf, SIX_CLASSIFIER_PATH+"RF_" + CV_fold_name + ".RFmod")
        
        # Save the feature importance vector
         
        # get a selector on the top 20 features
        selector = SelectFromModel(RFclf, prefit=True, threshold=-np.inf, max_features=20)
        del RFclf
        
        # project the training and the validation data
        sel_train_data=selector.transform(train_X)
        sel_valid_data=selector.transform(valid_X)
        
        #train a new SVC model on the reduced features
        from sklearn.svm import SVC

        # Create a classifier: a support vector RBF classifier

        SVCclf = SVC(C=1.0, kernel='rbf', gamma='scale',
                  probability=False, 
                  class_weight='balanced', 
                  decision_function_shape='ovo',
                  random_state = 0)

        SVCclf.fit(sel_train_data, train_y)
        
        # Get accuracy on training and validation data
        acc_sel_train = SVCclf.score(sel_train_data, train_y)
        acc_sel_val = SVCclf.score(sel_valid_data, valid_y)
        if verbose: 
            print("\t Reduced SVC model. Accuracy on train and valid: ", acc_sel_train, acc_sel_val)
        
        # save the model to disk
        joblib.dump(SVCclf, SIX_CLASSIFIER_PATH + "SVC_" + CV_fold_name + ".SVCmod")
        
        # Get prediction on all spectra
        pred = SVCclf.predict(sel_valid_data)
        
        # Gather SVC metrics
        if verbose:
            print("\t Computing the SVC model metrics.")
        SVCacc = balanced_accuracy_score(valid_y, pred)        
        SVCprec = precision_score(valid_y, pred, average = None)
        SVCrec = recall_score(valid_y, pred, average = None)
        SVCf1 = f1_score(valid_y, pred, average = None)
        SVCconf = confusion_matrix(valid_y, pred, normalize='true')
        SVCmetrics.append([SVCacc, SVCprec, SVCrec, SVCf1, SVCconf])
        
        if verbose: 
            print("\t SVC accuracy: ", SVCacc)
            print("\t SVC precision:", SVCprec)
            print("\t SVC recall:   ", SVCrec)
            print("\t SVC F1:       ", SVCf1)
            print("\t SVC confusion matrix:\n", SVCconf)          
        
        # Clear memory
        del SVCclf
        del train_X
        del train_y
        del valid_X
        del valid_y
        del sel_train_data
        del sel_valid_data
        gc.collect()
        
    return RFmetrics, SVCmetrics

In [6]:
# Read the pre-processed data and the tumor map of each sample
samples_0 = np.load(PREPROC_DATA_PATH, allow_pickle=True)
tumor_map_0 = np.load(TUMOR_MAP_PATH, allow_pickle=True)

samples = []
labels=[]

# Filter the data: retain only the tumor spots, based on the tumor map  
# Flatten the data (no spatial info needed here) 
# Skip the samples on the "ignore" list

for (sample, tumor) in zip(samples_0, tumor_map_0):
    if sample[SAMPLE_DICT_NAME_KEY] in ignore:
        print("Sample ignored:", sample[SAMPLE_DICT_NAME_KEY])
        continue    
    
    sample[SAMPLE_DICT_DATA_KEY] = sample[SAMPLE_DICT_DATA_KEY].reshape(-1, sample[SAMPLE_DICT_DATA_KEY].shape[2])
    tumor.resize(np.product(tumor.shape))
    sample[SAMPLE_DICT_DATA_KEY] = sample[SAMPLE_DICT_DATA_KEY][tumor == 1]
    samples.append(sample)
    
    # Set the methylation label 
    label = sample[SAMPLE_DICT_CLASS_ID_KEY]
    
    # All spots get as label the methylation label of the entire sample
    labels.append(label*np.ones(sample[SAMPLE_DICT_DATA_KEY].shape[0], dtype=int))
    
del samples_0
del tumor_map_0
gc.collect()

Sample ignored: HF-1887_via-t_2_1.h5_3


11

In [7]:
reset_seed(2022)
RFmetrics, SVCmetrics = CrossValidation(data=samples, labels=labels, CV_folds=CV5folds, verbose=True)

Validation fold 1
	 RF model. Accuracy on train and valid:  0.7163767824075211 0.2354011315247596
	 Computing the RF model metrics.
	 RF score:     0.2354011315247596
	 RF accuracy:  0.2323382614997261
	 RF precision: [0.004 0.743 0.029 0.266 0.229 0.102]
	 RF recall:    [0.002 0.41  0.011 0.338 0.243 0.39 ]
	 RF F1:        [0.002 0.528 0.015 0.298 0.236 0.161]
	 RF confusion matrix:
 [[0.002 0.001 0.003 0.142 0.852 0.   ]
 [0.076 0.41  0.257 0.034 0.209 0.014]
 [0.014 0.069 0.011 0.412 0.148 0.347]
 [0.02  0.012 0.013 0.338 0.434 0.182]
 [0.045 0.019 0.086 0.181 0.243 0.426]
 [0.001 0.06  0.    0.277 0.271 0.39 ]]
	 Reduced SVC model. Accuracy on train and valid:  0.6431402790944412 0.2824413746226585
	 Computing the SVC model metrics.
	 SVC accuracy:  0.3037162574790937
	 SVC precision: [0.747 0.74  0.055 0.227 0.251 0.071]
	 SVC recall:    [0.422 0.589 0.014 0.273 0.26  0.265]
	 SVC F1:        [0.539 0.656 0.022 0.248 0.255 0.112]
	 SVC confusion matrix:
 [[0.422 0.    0.001 0.209 0

In [8]:
del samples
del labels
gc.collect()

0

In [9]:
results = np.mean(np.asanyarray(RFmetrics,dtype=object), axis=0)
print("\t Mean RF accuracy: ", results[0])
print("\t Mean RF precision:", results[1])
print("\t Mean RF recall:   ", results[2])
print("\t Mean RF F1:       ", results[3])
print("\t Mean RF confusion matrix:\n", results[4])

	 Mean RF accuracy:  0.23966194081223788
	 Mean RF precision: [0.134 0.556 0.031 0.324 0.243 0.115]
	 Mean RF recall:    [0.091 0.477 0.02  0.391 0.183 0.275]
	 Mean RF F1:        [0.105 0.5   0.02  0.349 0.193 0.156]
	 Mean RF confusion matrix:
 [[0.091 0.263 0.049 0.122 0.282 0.192]
 [0.102 0.477 0.196 0.074 0.088 0.063]
 [0.065 0.396 0.02  0.257 0.099 0.162]
 [0.061 0.061 0.13  0.391 0.193 0.164]
 [0.13  0.078 0.055 0.34  0.183 0.214]
 [0.039 0.027 0.168 0.304 0.188 0.275]]


In [10]:
results = np.mean(np.asanyarray(SVCmetrics,dtype=object), axis=0)
print("\t Mean SVC accuracy: ", results[0])
print("\t Mean SVC precision:", results[1])
print("\t Mean SVC recall:   ", results[2])
print("\t Mean SVC F1:       ", results[3])
print("\t Mean SVC confusion matrix:\n", results[4])

	 Mean SVC accuracy:  0.26478264821583586
	 Mean SVC precision: [0.29  0.609 0.105 0.33  0.376 0.068]
	 Mean SVC recall:    [0.159 0.548 0.112 0.347 0.266 0.157]
	 Mean SVC F1:        [0.199 0.573 0.075 0.333 0.292 0.092]
	 Mean SVC confusion matrix:
 [[0.159 0.191 0.184 0.158 0.159 0.15 ]
 [0.073 0.548 0.192 0.051 0.091 0.045]
 [0.102 0.295 0.112 0.187 0.13  0.174]
 [0.1   0.117 0.052 0.347 0.217 0.167]
 [0.108 0.054 0.044 0.35  0.266 0.178]
 [0.086 0.038 0.218 0.303 0.199 0.157]]
